In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd workspace/theislab/mubind/notebooks/batch

/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/mubind/notebooks/batch


In [3]:
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '../../annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

/home/rio/miniconda3/envs/mubind/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


In [4]:
# motif = "ATT"
# x2, y2 = mb.datasets.simulate_xy(motif, n_trials=2100, seqlen=len(motif) + 1, max_mismatches=min(len(motif), 2), counts_size=1)
# x2, y2
# df = pd.DataFrame(data=y2)
# df.columns = [1]
# df.index = x2
# df[0] = 0
# df = df[[0, 1]]

In [16]:
df = mb.bindome.datasets.ProBound.ctcf(flank_length=0)
df = df.sort_values(1, ascending=False).reset_index(drop=True)
# data = df.head(1000)
# data = df.copy()

df.index = df['seq']
del df['seq']
df.index = df.index.astype(str).str[45:]

df = df[~df.index.duplicated(keep='first')]

data = df.sample(2500)
# data = mb.pp.sample_rounds(df, 2, 10000)

# remove as many nucleotides as posisble (faster training/convergence)


# data.index = data.index.astype(str).str[-15:]


In [17]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 5, 1

In [18]:
# TODO: skipped for now
n_rounds = 2

print('loading object (# entries)', data.shape[0])
dataset = mb.datasets.SelexDataset(data, n_rounds=n_rounds, labels=[0, 1])
train = tdata.DataLoader(dataset=dataset,
                         # batch_size=256,
                         batch_size=512,
                         shuffle=False)

loading object (# entries) 2500


In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
import numpy as np

In [21]:
%load_ext line_profiler


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [22]:
# %lprun -f mb.tl.train_network mb.tl.train_network(model, train, device, next_optimiser, criterion, num_epochs=20, early_stopping=100, log_each=2, dirichlet_regularization=0, exp_max=40, verbose=0)

In [23]:
# %lprun -f mb.tl.train_iterative mb.tl.train_iterative(train, device, w=18, show_logo=False, opt_kernel_shift=0, opt_kernel_length=0, dirichlet_regularization=dirichlet_regularization, lr=[0.01, 0.01], weight_decay=[0.01, 0.001], ignore_kernel=ignore_kernel, num_epochs=2, early_stopping=100, use_dinuc=False, n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None)

In [24]:
pd.set_option('display.expand_frame_repr', False)


In [25]:
# assign batch and data type

# data['batch'] = 0
# data['is_count_data'] = 1
# n_batches=3

dirichlet_regularization = 0
# for dirichlet_regularization_log in range(-5, 3):

n_epochs = 200
log_each = 100
n_kernels = 3
w = 15 # min(len(motif), 6)
lr = [0.01] * n_kernels
wd = [0.01,] + [0.001] * (n_kernels - 1)
early_stopping = [5,] + [25] * (n_kernels - 1)
opt_kernel_shift = 1
opt_kernel_length = 1

from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 1

import time
t0 = time.time()

criterion = mb.tl.PoissonLoss()

for ignore_kernel in [1]: # [0, 1]:
    print('\n\n##NEW JOB')
    print('ignore kernel', ignore_kernel)
    print('# trials', data.shape[0])
    model_by_k, res_next = mb.tl.optimize_iterative(train, device, w=w, show_logo=0, criterion=criterion,
                                                 opt_kernel_shift=opt_kernel_shift, opt_kernel_length=opt_kernel_length,
                                                 dirichlet_regularization=dirichlet_regularization, # 10 ** dirichlet_regularization_log,
                                                 lr=lr, weight_decay=wd, ignore_kernel=ignore_kernel, use_mono=True,
                                                 num_epochs=n_epochs, early_stopping=early_stopping, use_dinuc=True, # optimiser=torch.optim.LBFGS,
                                                 n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None) #  seed=seed) # seeds.index[0]) #
    print('##DONE....\n\n')



print('total time: %.3f s' % ((time.time() - t0)))
# res = []
# model_by_k = {}

model = model_by_k





##NEW JOB
ignore kernel 1
# trials 2500
next w 15 <class 'int'>
# rounds {2}
# rounds {2}
# use_mono True
# use_dinuc True
# batches 1
# kernels 3
# initial w 15
# enr_series True

Kernel to optimize 0

FREEZING KERNELS
optimizing feature type mono
setting grad status of kernel (mono, dinuc) at 0 to (1, 0)
setting grad status of kernel (mono, dinuc) at 1 to (0, 0)
setting grad status of kernel (mono, dinuc) at 2 to (0, 0)


kernels mask [0 1 1]
optimizer:  <class 'torch.optim.adam.Adam'> 
criterion: <class 'mubind.tl.loss.PoissonLoss'> 
# epochs: 200 
early_stopping: 5
lr= 0.01, weight_decay= 0.01, dir weight= 0
Epoch: 101, Loss: 0.843895 , best epoch: 99 secs per epoch: 0.053 s
Final loss: 0.8437027097
Total time (model/function): (10.550s / 10.550s)
Time per epoch (model/function): (0.053s/ 0.053s)
Time per epoch per 1k trials: 0.021s
Current time: 2022-12-25 11:56:47.017075
optimization of dinuc is not necessary for the intercepts (kernel=0). Skip...

Kernel to optimize 1

FREEZIN

KeyboardInterrupt: 

### We can visualize the overall results obtained by the network, once the training is finished

In [ ]:
import mubind as mb

In [ ]:
model = model_by_k
mb.tl.scores(model, train)


In [ ]:
rcParams['figure.figsize'] = 6, 6


In [ ]:
counts = mb.tl.predict(model, train)
counts

In [ ]:
rcParams['figure.figsize'] = 6, 6


In [ ]:
mb.pl.kmer_enrichment(model, train, log_scale=False, style='scatter', ylab='t1', xlab='p1')


In [ ]:
mb.pl.conv(model)


In [ ]:
import mubind as mb

In [ ]:
model = model_by_k
mb.tl.scores(model, train)


In [ ]:
counts = mb.tl.predict(model, train)
counts

In [ ]:
rcParams['figure.figsize'] = 6, 6

In [ ]:
train.dataset.rounds.shape

In [ ]:
model = model_by_k
mb.tl.scores(model, train)


In [ ]:
counts = mb.tl.predict(model, train)
counts

In [ ]:
mb.pl.kmer_enrichment(model, train, log_scale=False, style='scatter', ylab='t1', xlab='p1')


In [ ]:
mb.pl.conv_mono(model)


In [ ]:
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '../../annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

In [ ]:
import itertools

def _mismatch(word, letters, num_mismatches):
    for locs in itertools.combinations(range(len(word)), num_mismatches):
        this_word = [[char] for char in word]
        for loc in locs:
            orig_char = word[loc]
            this_word[loc] = [l for l in letters if l != orig_char]
        for poss in itertools.product(*this_word):
            yield "".join(poss)



In [ ]:
# motif = "ATT"
# x2, y2 = mb.datasets.simulate_xy(motif, n_trials=2100, seqlen=len(motif) + 1, max_mismatches=min(len(motif), 2), counts_size=1)
# x2, y2
# df = pd.DataFrame(data=y2)
# df.columns = [1]
# df.index = x2
# df[0] = 0
# df = df[[0, 1]]

In [ ]:
df = mb.bindome.datasets.ProBound.ctcf(flank_length=0)
df = df.sort_values(1, ascending=False).reset_index(drop=True)
# data = df.head(1000)
# data = df.copy()
data = df.sample(10000, random_state=50)
# data = df.sample(df.shape[0], random_state=50)
# data = df.copy()
data.index = data['seq']

# remove as many nucleotides as posisble (faster training/convergence)
data.index = data.index.astype(str).str[45:]


# data.index = data.index.astype(str).str[-15:]
del data['seq']

In [ ]:
data.shape

In [ ]:
data = data.sample(data.shape[0], random_state=100)

In [ ]:
data.shape[0]

In [ ]:
data.shape

In [ ]:
# # define batches
# batch = np.random.randint(2, size=data.shape[0])

In [ ]:
# # data[batch == 0] = data[batch == 0] + 20
# # data[batch == 1] = data[batch == 1] + 10
# data['batch'] = batch
#
# data = data[data['batch'] == 1]

In [ ]:


from matplotlib import rcParams
rcParams['figure.figsize'] = 5, 1

In [ ]:
# TODO: skipped for now
n_rounds = 2

print('loading object (# entries)', data.shape[0])
dataset = mb.datasets.SelexDataset(data, n_rounds=n_rounds, labels=[0, 1])
train = tdata.DataLoader(dataset=dataset,
                         # batch_size=256,
                         batch_size=512,
                         shuffle=False)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np

In [ ]:
%load_ext line_profiler


In [ ]:
# %lprun -f mb.tl.train_network mb.tl.train_network(model, train, device, next_optimiser, criterion, num_epochs=20, early_stopping=100, log_each=2, dirichlet_regularization=0, exp_max=40, verbose=0)

In [ ]:
# %lprun -f mb.tl.train_iterative mb.tl.train_iterative(train, device, w=18, show_logo=False, opt_kernel_shift=0, opt_kernel_length=0, dirichlet_regularization=dirichlet_regularization, lr=[0.01, 0.01], weight_decay=[0.01, 0.001], ignore_kernel=ignore_kernel, num_epochs=2, early_stopping=100, use_dinuc=False, n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None)

In [ ]:
# assign batch and data type

# data['batch'] = 0
# data['is_count_data'] = 1
# n_batches=3

dirichlet_regularization = 0
# for dirichlet_regularization_log in range(-5, 3):

n_epochs = 100
log_each = 10
n_kernels = 4
lr = [0.01] * n_kernels
wd = [0.01,] + [0.001] * (n_kernels - 1)
early_stopping = [5,] + [25] * (n_kernels - 1)
w = 15 # min(len(motif), 6)
opt_kernel_shift = 1
opt_kernel_length = 1

from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 1

import time
t0 = time.time()

criterion = mb.tl.PoissonLoss()

for ignore_kernel in [1]: # [0, 1]:
    print('\n\n##NEW JOB')
    print('ignore kernel', ignore_kernel)
    model_by_k, res_next = mb.tl.optimize_iterative(train, device, w=w, show_logo=0, criterion=criterion,
                                                 opt_kernel_shift=opt_kernel_shift, opt_kernel_length=opt_kernel_length,
                                                 dirichlet_regularization=dirichlet_regularization, # 10 ** dirichlet_regularization_log,
                                                 lr=wd, weight_decay=wd, ignore_kernel=ignore_kernel, use_mono=True,
                                                 num_epochs=n_epochs, early_stopping=early_stopping, use_dinuc=True, # optimiser=torch.optim.LBFGS,
                                                 n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None) #  seed=seed) # seeds.index[0]) #
    print('##DONE....\n\n')


print('total time: %.3f s' % ((time.time() - t0)))
# res = []
# model_by_k = {}

model = model_by_k



### We can visualize the overall results obtained by the network, once the training is finished

In [ ]:
import mubind as mb

In [ ]:
model = model_by_k
mb.tl.scores(model, train)


In [ ]:
counts = mb.tl.predict(model, train)
counts

In [ ]:
rcParams['figure.figsize'] = 6, 6

In [ ]:
train.dataset.rounds.shape

In [ ]:
mb.pl.kmer_enrichment(model, train, log_scale=False, style='scatter', ylab='t1', xlab='p1', k=9)


In [ ]:
mb.pl.conv(model)
